In [2]:
# Python code for Pollard p-1 Factorization Method
# Based on https://github.com/Ganapati/RsaCtfTool/blob/c13713a2808a03b15eb62e35605b9eb4271069cc/attacks/single_key/pollard_p_1.py

import binascii
import gmpy2
import math
from tqdm import tqdm


def _primes_yield_gmpy(n):
    p = i = 1
    while i <= n:
        p = gmpy2.next_prime(p)
        yield p
        i += 1


def primes(n):
    return list(_primes_yield_gmpy(n))


def pollard_P_1(n, progress=True, num_primes=2000):
    """Pollard P1 implementation"""
    z = []
    logn = math.log(int(gmpy2.isqrt(n)))
    prime = primes(num_primes)

    for j in range(0, len(prime)):
        primej = prime[j]
        logp = math.log(primej)
        for i in range(1, int(logn / logp) + 1):
            z.append(primej)

    try:
        for pp in tqdm(prime, disable=(not progress)):
            i = 0
            x = pp
            while 1:
                x = gmpy2.powmod(x, z[i], n)
                i = i + 1
                y = gmpy2.gcd(n, x - 1)
                if y != 1:
                    p = y
                    q = n // y
                    return p, q
                if i >= len(z):
                    return 0, None
    except TypeError:
        return 0, None


e = 0x10001
c = 0xa913a96e215b5aa79c702d27fa375c73d06787639c4131fb32877cafefaa8faf70e15f6a17ef2a9a6f5310b157cb287b740e77cb5385081d1853a9104bc16357b259fa2d146bd87398d4ef6f1c078289812952c67792cf9cd745049aeb9d4ab4dff2825a9c0b3381f19b2a67164f9d4de33c25f98bc2f224feb5507b531e1a1c7be5ed2d8ddd01f3fae37245e8cf99c75a21848993d445e1d6d69d555a3e6cc8055704fdde88df9084bda3ea65a9384fa64bf8df4d88946449526320c15d4d2d871638070489adf3f8c95caffeab40b0d137a9319be20cdf6ebbaf037f62093d9bd33edd4ffd7e1929b9ab06252956fd85250a0515ef2b4e035017be5702cdd3
n = 0xb03ea698ce2b51fb00e11e6fbaf1e5373dc5b0c70eb2b14a36d21e8667be8774eee51f6050a10237f6b24f21204fc8013681e7ed72ed051188f3274aae8f1de0d39389b514c196fa82c98a270bfabefd044da8c687b0e114ebbde82536c0709ac5ad81bfe0077e9d9b798ad5abecee52767e68f8060c45936521fd93893102eb1676f2ff41324a7a6b3dff2e830538e06d25934e9f14bf6b40ab5674fe648e314bf06f84282f5ef52bc1401de3a42eb66e64bcdadd2674348e5bdb7016feda44d719af387a948ad81cbaed10213dd930fc7bc7677d8c4cdab0645d0ff15e6ad6ca37135942c3be08f23e7be0992c8b3370dcdc31045e086d823107fb2e443dc9
num_primes = 7_000
p, q = pollard_P_1(n, num_primes=num_primes)
print(p)
print(q)
if q is None:
    print("Pollard p-1 Factorization Attack Failed. You can try increasing `num_primes`...")
else:
    n = p * q

    m = gmpy2.lcm(p - 1, q - 1)
    d = pow(e, -1, m)

    m = pow(c, d, n)
    print("Flag: %s" % binascii.unhexlify(hex(m)[2:]).decode())

  0%|          | 0/7000 [00:16<?, ?it/s]

166778678181685622576528273627655558408416154535903271612099666211650718813574353644982679332718906416639262976090347767411961612927781793104151632976133030663080503056699508483393432483211221435369665266139037807475222869716828564531056095553338227197069928344760597134918054695465937470622255206477029220627
133403359244090432282498741122238666960102072347999139399194192684930601046915449960642287092839790447663496118648044185489682661151904512440107598783041356688524538695035027282386056258524355032597865645378353170572838674473248880702185307449067157154593351616151961260356975594350933975792987875901590674739
Flag: picoCTF{p0ll4rd_f4ct0r1z4at10n_FTW_376ebfe7}
